In [5]:
import pymorton as pm
from random import randrange
import numpy as np
import zCurve as zC
from itertools import permutations

def coordinate_permutations(coord):
    """
    Given a coordinate (tuple or list) of length D,
    return all D! permutations as tuples.
    """
    return list(permutations(coord))


In [4]:
from random import randrange

bit_size = 16
max_val = 2**bit_size - 1
no_samples = 10**6
data_points = [(randrange(0, max_val), randrange(0, max_val), randrange(0, max_val)) for i in range(no_samples)]

In [13]:
import zCurve as z 
morton_codes = z.par_interlace(data_points, dims=3, bits_per_dim=16)
#data_points == z.par_deinterlace(morton_codes, dims=3)
len(morton_codes)


1000000

In [ ]:
N=2

arr=np.linspace(1,48,48).reshape((3,4,4)).astype(np.uint8)
arr=np.ones((N,N,N)).astype(np.uint8)
dims = arr.ndim
Nx, Ny, Nz = arr.shape
bits_per_dim=int(np.log2(max(Nx, Ny, Nz)))
Npermutations = np.math.factorial(dims)
#print(arr)

# choose uint type based on bits_per_dim
if bits_per_dim * dims <= 8:
    dtype = np.uint8
elif bits_per_dim * dims <= 16:
    dtype = np.uint16
elif bits_per_dim * dims <= 32:
    dtype = np.uint32
else:
    dtype = np.uint64

print(dtype)

# Compute Morton codes for each voxel
codes1 = np.empty((Nx, Ny, Nz, Npermutations), dtype=dtype)
codes = np.empty((Nx, Ny, Nz, Npermutations), dtype=dtype)
for x in range(Nx):
    for y in range(Ny):
        for z in range(Nz):
          #  codes1[x, y, z] = pm.interleave3(x, y, z)
            permutation_list = coordinate_permutations((x, y, z))
          #  zperms = zC.par_interlace(permutation_list, dims=3, bits_per_dim=bits_per_dim)
            permutation_list = [(x,y,z), (x,z,y), (y,x,z), (y,z,x), (z,x,y), (z,y,x)]
            codes[x, y, z, 0] = zC.interlace(x, y, z, dims=3, bits_per_dim=bits_per_dim)
         #   codes[x, y, z, 1] = zC.interlace(x, z, y, dims=3, bits_per_dim=bits_per_dim)
            print(type(x), type(y), type(z), type(permutation_list[0][0]))
            codes[x, y, z, :] = zC.par_interlace(permutation_list, dims=3, bits_per_dim=bits_per_dim)
            print(codes[x, y, z, :])
          #  print(codes[x, y, z, 0], codes[x, y, z, 1])

#print(codes)
# Flatten both arrays and sort by Morton code
flat_arr = arr.ravel()
flat_codes = [codes[..., i].ravel() for i in range(Npermutations)]
orders = [np.argsort(flat_codes[i]) for i in range(Npermutations)]
zorders = [flat_arr[orders[i]] for i in range(Npermutations)]
#order = np.argsort(flat_codes)
#order1 = np.argsort(codes1.ravel())
#z_ordered = flat_arr[order]
#print(order), print(flat_arr[order1 ]), print(np.max(order)), print("O", len(order), np.unique(order).shape)
orders

In [25]:

def all_permutation_codes_listmode_3D(Nx, Ny, Nz, bits_per_dim, dtype=np.uint64):
    """
    Compute Morton codes for every (x, y, z) voxel for all 3! coordinate permutations.
    Uses zC.par_interlace(), which expects a list of tuples.
    """
    D = 3
    perms = list(permutations(range(D)))
    Nperm = len(perms)

    # Make coordinate grid (Nx, Ny, Nz, 3)
    x, y, z = np.meshgrid(
        np.arange(Nx, dtype=dtype),
        np.arange(Ny, dtype=dtype),
        np.arange(Nz, dtype=dtype),
        indexing="ij"
    )
    coords = np.stack((x, y, z), axis=-1).reshape(-1, D)  # (Npoints, 3)

    # Convert to list of tuples once
    coord_list = [tuple(c) for c in coords]  # e.g. [(0,0,0), (0,0,1), ...]

    # Preallocate output
    codes_flat = np.empty((len(coord_list), Nperm), dtype=dtype)

    # Process all 6 permutations — each with one call to par_interlace
    for i, p in enumerate(perms):
        # reorder coordinates according to permutation
        permuted_list = [(int(c[p[0]]), int(c[p[1]]), int(c[p[2]])) for c in coord_list]
      
       # zvals=zC.par_interlace(permuted_list, dims=D, bits_per_dim=bits_per_dim)
        codes_flat[:, i] = zC.par_interlace(permuted_list, dims=D, bits_per_dim=bits_per_dim)

    orders = [np.argsort(codes_flat[:, i]).astype(dtype) for i in range(Nperm)]
    return orders

Nx, Ny, Nz = 2,6,6
bits_per_dim = max(Nx, Ny, Nz).bit_length()+1
print(bits_per_dim)
codes = all_permutation_codes_listmode_3D(Nx, Ny, Nz, bits_per_dim=bits_per_dim, dtype=np.uint32)
codes

4


[array([ 0, 36,  6, 42,  1, 37,  7, 43, 12, 48, 18, 54, 13, 49, 19, 55,  2,
        38,  8, 44,  3, 39,  9, 45, 14, 50, 20, 56, 15, 51, 21, 57, 24, 60,
        30, 66, 25, 61, 31, 67, 26, 62, 32, 68, 27, 63, 33, 69,  4, 40, 10,
        46,  5, 41, 11, 47, 16, 52, 22, 58, 17, 53, 23, 59, 28, 64, 34, 70,
        29, 65, 35, 71], dtype=uint32),
 array([ 0, 36,  1, 37,  6, 42,  7, 43,  2, 38,  3, 39,  8, 44,  9, 45, 12,
        48, 13, 49, 18, 54, 19, 55, 14, 50, 15, 51, 20, 56, 21, 57,  4, 40,
         5, 41, 10, 46, 11, 47, 16, 52, 17, 53, 22, 58, 23, 59, 24, 60, 25,
        61, 30, 66, 31, 67, 26, 62, 27, 63, 32, 68, 33, 69, 28, 64, 29, 65,
        34, 70, 35, 71], dtype=uint32),
 array([ 0,  6, 36, 42,  1,  7, 37, 43, 12, 18, 48, 54, 13, 19, 49, 55,  2,
         8, 38, 44,  3,  9, 39, 45, 14, 20, 50, 56, 15, 21, 51, 57, 24, 30,
        60, 66, 25, 31, 61, 67, 26, 32, 62, 68, 27, 33, 63, 69,  4, 10, 40,
        46,  5, 11, 41, 47, 16, 22, 52, 58, 17, 23, 53, 59, 28, 34, 64, 70,
        

In [ ]:
import numpy as np
from itertools import permutations

def precompute_zcurves(Ndims, bits_per_dim, dtype=np.uint64):
    """
    Compute Morton codes for every voxel coordinate in D dimensions,
    for all D! permutations of coordinate ordering.

    Parameters
    ----------
    Ndims : tuple of ints
        The side lengths along each dimension, e.g. (Nx, Ny, Nz) or (N0, N1, ..., N_{D-1}).
    bits_per_dim : int
        Number of bits allocated per dimension for Morton encoding.
    dtype : numpy dtype (default: np.uint64)

    Returns
    -------
    orders : list of 1D numpy arrays (dtype=dtype)
        For each permutation of coordinate axes, an ordering (argsort) of Morton codes.
    """
    D = len(Ndims)
    perms = list(permutations(range(D)))
    Nperm = len(perms)

    # Create a coordinate grid
    grids = np.meshgrid(*[np.arange(n, dtype=dtype) for n in Ndims], indexing='ij')
    # Stack into shape (Npoints, D)
    coords = np.stack(grids, axis=-1).reshape(-1, D)

    # Convert to tuple list once, because par_interlace expects list[(coord tuple)...]
    coord_list = [tuple(int(c[d]) for d in range(D)) for c in coords]

    # Allocate Morton codes array: one column per permutation
    codes_flat = np.empty((len(coord_list), Nperm), dtype=dtype)

    # Compute Morton codes for each permutation
    for i, p in enumerate(perms):
        permuted_list = [tuple(c[pj] for pj in p) for c in coord_list]
        codes_flat[:, i] = zC.par_interlace(permuted_list, dims=D, bits_per_dim=bits_per_dim)

    # Convert codes to sort orders
    orders = [np.argsort(codes_flat[:, i]).astype(dtype) for i in range(Nperm)]
    return orders

Nt, Nx, Ny, Nz = 1,4,4,4
bits_per_dim = max(Nt, Nx, Ny, Nz).bit_length()+1
print(bits_per_dim)
orders = precompute_zcurves((Nt,Nx, Ny, Nz), bits_per_dim=bits_per_dim, dtype=np.uint32)

4


In [37]:
arr=np.linspace(1,Nt*Nx*Ny*Nz,Nt*Nx*Ny*Nz).reshape((Nt,Nx,Ny,Nz)).astype(np.uint32)
flat_arr = arr.ravel()
zorders = [flat_arr[orders[i]] for i in range(len(orders))]
zorders

[array([ 1, 17,  5, 21,  2, 18,  6, 22, 33, 49, 37, 53, 34, 50, 38, 54,  9,
        25, 13, 29, 10, 26, 14, 30, 41, 57, 45, 61, 42, 58, 46, 62,  3, 19,
         7, 23,  4, 20,  8, 24, 35, 51, 39, 55, 36, 52, 40, 56, 11, 27, 15,
        31, 12, 28, 16, 32, 43, 59, 47, 63, 44, 60, 48, 64], dtype=uint32),
 array([ 1, 17,  2, 18,  5, 21,  6, 22, 33, 49, 34, 50, 37, 53, 38, 54,  3,
        19,  4, 20,  7, 23,  8, 24, 35, 51, 36, 52, 39, 55, 40, 56,  9, 25,
        10, 26, 13, 29, 14, 30, 41, 57, 42, 58, 45, 61, 46, 62, 11, 27, 12,
        28, 15, 31, 16, 32, 43, 59, 44, 60, 47, 63, 48, 64], dtype=uint32),
 array([ 1,  5, 17, 21,  2,  6, 18, 22,  9, 13, 25, 29, 10, 14, 26, 30, 33,
        37, 49, 53, 34, 38, 50, 54, 41, 45, 57, 61, 42, 46, 58, 62,  3,  7,
        19, 23,  4,  8, 20, 24, 11, 15, 27, 31, 12, 16, 28, 32, 35, 39, 51,
        55, 36, 40, 52, 56, 43, 47, 59, 63, 44, 48, 60, 64], dtype=uint32),
 array([ 1,  5,  2,  6, 17, 21, 18, 22,  9, 13, 10, 14, 25, 29, 26, 30,  3,
         7, 

In [38]:
arr

array([[[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12],
         [13, 14, 15, 16]],

        [[17, 18, 19, 20],
         [21, 22, 23, 24],
         [25, 26, 27, 28],
         [29, 30, 31, 32]],

        [[33, 34, 35, 36],
         [37, 38, 39, 40],
         [41, 42, 43, 44],
         [45, 46, 47, 48]],

        [[49, 50, 51, 52],
         [53, 54, 55, 56],
         [57, 58, 59, 60],
         [61, 62, 63, 64]]]], dtype=uint32)